In [4]:
from spikingjelly.datasets.cifar10_dvs import CIFAR10DVS
from spikingjelly.datasets.dvs128_gesture import DVS128Gesture

print('CIFAR10-DVS downloadable', CIFAR10DVS.downloadable())
print('resource, url, md5/n', CIFAR10DVS.resource_url_md5())

print('DVS128Gesture downloadable', DVS128Gesture.downloadable())
print('resource, url, md5/n', DVS128Gesture.resource_url_md5())

CIFAR10-DVS downloadable True
resource, url, md5/n [('airplane.zip', 'https://ndownloader.figshare.com/files/7712788', '0afd5c4bf9ae06af762a77b180354fdd'), ('automobile.zip', 'https://ndownloader.figshare.com/files/7712791', '8438dfeba3bc970c94962d995b1b9bdd'), ('bird.zip', 'https://ndownloader.figshare.com/files/7712794', 'a9c207c91c55b9dc2002dc21c684d785'), ('cat.zip', 'https://ndownloader.figshare.com/files/7712812', '52c63c677c2b15fa5146a8daf4d56687'), ('deer.zip', 'https://ndownloader.figshare.com/files/7712815', 'b6bf21f6c04d21ba4e23fc3e36c8a4a3'), ('dog.zip', 'https://ndownloader.figshare.com/files/7712818', 'f379ebdf6703d16e0a690782e62639c3'), ('frog.zip', 'https://ndownloader.figshare.com/files/7712842', 'cad6ed91214b1c7388a5f6ee56d08803'), ('horse.zip', 'https://ndownloader.figshare.com/files/7712851', 'e7cbbf77bec584ffbf913f00e682782a'), ('ship.zip', 'https://ndownloader.figshare.com/files/7712836', '41c7bd7d6b251be82557c6cce9a7d5c9'), ('truck.zip', 'https://ndownloader.figs

In [5]:
from spikingjelly.datasets.dvs128_gesture import DVS128Gesture

root_dir = 'D:/MIND/MIND/materials/lab/datasets/DVS128Gesture'
train_set = DVS128Gesture(root_dir, train=True, data_type='event')

The [D:/MIND/MIND/materials/lab/datasets/DVS128Gesture\download] directory for saving downloaded files already exists, check files...
The file [D:/MIND/MIND/materials/lab/datasets/DVS128Gesture\download\DvsGesture.tar.gz] does not exist or is corrupted.


NotImplementedError: This dataset can not be downloaded by SpikingJelly, please download [DvsGesture.tar.gz] from [https://ibm.ent.box.com/s/3hiq58ww1pbbjrinh367ykfdf60xsfm8/folder/50167556794] manually and put files at D:/MIND/MIND/materials/lab/datasets/DVS128Gesture\download.